In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping

path = "dataset/normalized/"

df = pd.read_csv(path + "kart.csv")

# Split the data into features (X) and target (y)
X = df.drop('action', axis=1)
y = df['action']

# Split the data into training and test sets
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)

# Número de clases
num_classes = 8

# Convertir a codificación one-hot para usar MSE
y_train = tf.keras.utils.to_categorical(y_train_encoded, num_classes=num_classes, dtype='float32')
y_val = tf.keras.utils.to_categorical(y_val_encoded, num_classes=num_classes, dtype='float32')

early_stopping = EarlyStopping(
    monitor='val_loss',  # Métrica a monitorear (puede ser 'val_accuracy', 'val_loss', etc.)
    patience=10,          # Número de épocas sin mejora antes de detener el entrenamiento
    restore_best_weights=True  # Restaura los pesos del modelo al mejor logrado durante el entrenamiento
)

model = keras.Sequential()

model.add(layers.Dense(64, activation='relu', input_shape=(8,)))

# Agrega una capa de Reshape para agregar la dimensión de los pasos de tiempo
model.add(layers.Reshape((1, 64)))

# Capa LSTM
model.add(layers.LSTM(64))

# Capa de salida
model.add(layers.Dense(num_classes, activation='linear'))

model.summary()

model.compile(
    loss=keras.losses.MeanSquaredError(),
    optimizer="sgd",
    metrics=["accuracy"],
)

history = model.fit(
    x_train,
    y_train,
    batch_size=16,
    epochs=5000,
    validation_data=(x_val, y_val),
    callbacks=[early_stopping]
)



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                576       
                                                                 
 reshape (Reshape)           (None, 1, 64)             0         
                                                                 
 lstm (LSTM)                 (None, 64)                33024     
                                                                 
 dense_1 (Dense)             (None, 8)                 520       
                                                                 
Total params: 34120 (133.28 KB)
Trainable params: 34120 (133.28 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________

Epoch 1/5000


369/369 [==============================] - 3s 4ms/step - loss: 0.0922 - accuracy: 0.4132 - val_loss: 0.0750 - val_accuracy: 0.5671
E

369/369 [==============================] - 1s 2ms/step - loss: 0.0564 - accuracy: 0.6340 - val_loss: 0.0559 - val_accuracy: 0.6714
Epoch 44/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0562 - accuracy: 0.6357 - val_loss: 0.0557 - val_accuracy: 0.6734
Epoch 45/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0561 - accuracy: 0.6394 - val_loss: 0.0555 - val_accuracy: 0.6741
Epoch 46/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0559 - accuracy: 0.6402 - val_loss: 0.0553 - val_accuracy: 0.6755
Epoch 47/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0558 - accuracy: 0.6418 - val_loss: 0.0552 - val_accuracy: 0.6782
Epoch 48/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0556 - accuracy: 0.6460 - val_loss: 0.0550 - val_accuracy: 0.6782
Epoch 49/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0555 - accuracy: 0.6480 - val_loss: 0.0548 - val_accuracy: 0.6795

369/369 [==============================] - 1s 2ms/step - loss: 0.0517 - accuracy: 0.6904 - val_loss: 0.0507 - val_accuracy: 0.7087
Epoch 100/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0516 - accuracy: 0.6899 - val_loss: 0.0507 - val_accuracy: 0.7060
Epoch 101/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0516 - accuracy: 0.6916 - val_loss: 0.0506 - val_accuracy: 0.7060
Epoch 102/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0516 - accuracy: 0.6904 - val_loss: 0.0506 - val_accuracy: 0.7060
Epoch 103/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0515 - accuracy: 0.6917 - val_loss: 0.0506 - val_accuracy: 0.7066
Epoch 104/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0515 - accuracy: 0.6926 - val_loss: 0.0505 - val_accuracy: 0.7066
Epoch 105/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0514 - accuracy: 0.6919 - val_loss: 0.0505 - val_accuracy: 

369/369 [==============================] - 1s 2ms/step - loss: 0.0498 - accuracy: 0.7033 - val_loss: 0.0490 - val_accuracy: 0.7114
Epoch 156/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0498 - accuracy: 0.7019 - val_loss: 0.0489 - val_accuracy: 0.7188
Epoch 157/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0498 - accuracy: 0.7034 - val_loss: 0.0489 - val_accuracy: 0.7121
Epoch 158/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0497 - accuracy: 0.6980 - val_loss: 0.0489 - val_accuracy: 0.7127
Epoch 159/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0497 - accuracy: 0.7028 - val_loss: 0.0489 - val_accuracy: 0.7175
Epoch 160/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0497 - accuracy: 0.7019 - val_loss: 0.0489 - val_accuracy: 0.7148
Epoch 161/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0496 - accuracy: 0.7028 - val_loss: 0.0488 - val_accuracy: 

369/369 [==============================] - 1s 2ms/step - loss: 0.0486 - accuracy: 0.7039 - val_loss: 0.0480 - val_accuracy: 0.7195
Epoch 212/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0485 - accuracy: 0.7039 - val_loss: 0.0480 - val_accuracy: 0.7209
Epoch 213/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0485 - accuracy: 0.7034 - val_loss: 0.0480 - val_accuracy: 0.7209
Epoch 214/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0485 - accuracy: 0.7029 - val_loss: 0.0480 - val_accuracy: 0.7215
Epoch 215/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0485 - accuracy: 0.7031 - val_loss: 0.0479 - val_accuracy: 0.7188
Epoch 216/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0485 - accuracy: 0.7034 - val_loss: 0.0479 - val_accuracy: 0.7209
Epoch 217/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0484 - accuracy: 0.7029 - val_loss: 0.0479 - val_accuracy: 

369/369 [==============================] - 1s 2ms/step - loss: 0.0477 - accuracy: 0.7087 - val_loss: 0.0474 - val_accuracy: 0.7263
Epoch 268/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0477 - accuracy: 0.7063 - val_loss: 0.0474 - val_accuracy: 0.7249
Epoch 269/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0476 - accuracy: 0.7078 - val_loss: 0.0474 - val_accuracy: 0.7276
Epoch 270/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0476 - accuracy: 0.7087 - val_loss: 0.0474 - val_accuracy: 0.7276
Epoch 271/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0476 - accuracy: 0.7051 - val_loss: 0.0474 - val_accuracy: 0.7297
Epoch 272/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0476 - accuracy: 0.7075 - val_loss: 0.0474 - val_accuracy: 0.7283
Epoch 273/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0476 - accuracy: 0.7077 - val_loss: 0.0474 - val_accuracy: 

369/369 [==============================] - 1s 2ms/step - loss: 0.0470 - accuracy: 0.7114 - val_loss: 0.0470 - val_accuracy: 0.7270
Epoch 324/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0470 - accuracy: 0.7128 - val_loss: 0.0470 - val_accuracy: 0.7290
Epoch 325/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0470 - accuracy: 0.7124 - val_loss: 0.0470 - val_accuracy: 0.7324
Epoch 326/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0470 - accuracy: 0.7104 - val_loss: 0.0470 - val_accuracy: 0.7310
Epoch 327/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0470 - accuracy: 0.7165 - val_loss: 0.0469 - val_accuracy: 0.7283
Epoch 328/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0469 - accuracy: 0.7156 - val_loss: 0.0469 - val_accuracy: 0.7297
Epoch 329/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0469 - accuracy: 0.7141 - val_loss: 0.0469 - val_accuracy: 

369/369 [==============================] - 1s 2ms/step - loss: 0.0465 - accuracy: 0.7121 - val_loss: 0.0468 - val_accuracy: 0.7222
Epoch 380/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0465 - accuracy: 0.7138 - val_loss: 0.0467 - val_accuracy: 0.7290
Epoch 381/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0465 - accuracy: 0.7160 - val_loss: 0.0467 - val_accuracy: 0.7297
Epoch 382/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0465 - accuracy: 0.7150 - val_loss: 0.0467 - val_accuracy: 0.7290
Epoch 383/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0465 - accuracy: 0.7143 - val_loss: 0.0467 - val_accuracy: 0.7290
Epoch 384/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0465 - accuracy: 0.7131 - val_loss: 0.0467 - val_accuracy: 0.7304
Epoch 385/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0465 - accuracy: 0.7145 - val_loss: 0.0467 - val_accuracy: 

369/369 [==============================] - 1s 2ms/step - loss: 0.0461 - accuracy: 0.7141 - val_loss: 0.0465 - val_accuracy: 0.7256
Epoch 436/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0461 - accuracy: 0.7167 - val_loss: 0.0465 - val_accuracy: 0.7249
Epoch 437/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0460 - accuracy: 0.7187 - val_loss: 0.0464 - val_accuracy: 0.7290
Epoch 438/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0460 - accuracy: 0.7190 - val_loss: 0.0464 - val_accuracy: 0.7304
Epoch 439/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0460 - accuracy: 0.7197 - val_loss: 0.0464 - val_accuracy: 0.7276
Epoch 440/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0460 - accuracy: 0.7141 - val_loss: 0.0464 - val_accuracy: 0.7256
Epoch 441/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0460 - accuracy: 0.7184 - val_loss: 0.0464 - val_accuracy: 

369/369 [==============================] - 1s 2ms/step - loss: 0.0456 - accuracy: 0.7216 - val_loss: 0.0463 - val_accuracy: 0.7215
Epoch 492/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0456 - accuracy: 0.7221 - val_loss: 0.0462 - val_accuracy: 0.7297
Epoch 493/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0456 - accuracy: 0.7226 - val_loss: 0.0462 - val_accuracy: 0.7304
Epoch 494/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0456 - accuracy: 0.7194 - val_loss: 0.0462 - val_accuracy: 0.7270
Epoch 495/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0456 - accuracy: 0.7231 - val_loss: 0.0462 - val_accuracy: 0.7256
Epoch 496/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0456 - accuracy: 0.7231 - val_loss: 0.0461 - val_accuracy: 0.7304
Epoch 497/5000
369/369 [==============================] - 1s 2ms/step - loss: 0.0456 - accuracy: 0.7251 - val_loss: 0.0461 - val_accuracy: 